<a href="https://colab.research.google.com/github/finardi/tutos/blob/master/Triplet-CNNdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence-transformers

     |████████████████████████████████| 71kB 3.6MB/s 
     |████████████████████████████████| 778kB 13.8MB/s 
     |████████████████████████████████| 1.1MB 52.6MB/s 
     |████████████████████████████████| 890kB 50.9MB/s 
     |████████████████████████████████| 3.0MB 48.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.5.1-cp36-none-any.whl size=100387 sha256=d5c7738067fb9aa6e83a304e295097c13b04b27c2aca52d80d640eb1ef5ce92c
  Stored in directory: /root/.cache/pip/wheels/1a/05/25/5201c408b8048c9663d75c82dc079cb72633cf543af1aa84da
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=046894ffe8446ed9f7c8900b145a131f17ec572ca2a586ecd7cf1009c63c631a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [2]:
import tensorflow_datasets as tfds
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sentence_transformers import SentenceTransformer, util

#Load summarization dataset formed of content and summarization

In [3]:
def load_evaluation_data(load_num=1000, split="train"):
    cnn_data = tfds.load("cnn_dailymail")
    
    if load_num is not None:
        data = cnn_data[split].take(load_num)
    else:
        data = cnn_data[split]

    data_np_generator = tfds.as_numpy(data)

    article=[]
    highlights=[]

    for i in data_np_generator:
    #     print(i)
        article.append(i["article"].decode("utf-8"))
        highlights.append(i["highlights"].decode("utf-8"))
        
    return article, highlights

In [4]:
article, target = load_evaluation_data(load_num=200_000)

INFO:absl:No config specified, defaulting to first: cnn_dailymail/plain_text
INFO:absl:Load pre-computed datasetinfo (eg: splits) from bucket.
INFO:absl:Loading info from GCS for cnn_dailymail/plain_text/3.0.0
INFO:absl:Generating dataset cnn_dailymail (/root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0)


INFO:absl:Downloading https://drive.google.com/uc?export=download&id=0BwmD_VLjROrfTHk4NFg2SndKcjQ into /root/tensorflow_datasets/downloads/ucexport_download_id_0BwmD_VLjROrfTHk4NFg2SndKG8BdJPpt2iRo6Dpzz23CByJuAePEilB-pxbcBCHaWDs.tmp.f3309b4f39234949826d50cfe88c50b8...
INFO:absl:Downloading https://drive.google.com/uc?export=download&id=0BwmD_VLjROrfM1BxdkxVaTY2bWs into /root/tensorflow_datasets/downloads/ucexport_download_id_0BwmD_VLjROrfM1BxdkxVaTY2zVV-G71RIXPssrrvSAjt19Cy91r-9CQ2F9DMKA0uFk0.tmp.4e2fc9c8928d44ca84aacd6b9a9221a3...
INFO:absl:Downloading https://raw.githubusercontent.com/abisee/cnn-dailymail/master/url_lists/all_test.txt into /root/tensorflow_datasets/downloads/raw.gith.com_abis_cnn-dail_mast_url_list_a705_isK790OHOPsIZX-ACsObq_vchU9r5Uduh6ULX3c.txt.tmp.d6d42b7dd64c4fd0a0e0dfad8a913717...
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advise

INFO:absl:Generating split train


Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteN53DWB/cnn_dailymail-train.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteN53DWB/cnn_dailymail-train.tfrecord. Shard lengths: [17945, 17944, 17945, 17944, 17945, 17944, 17945, 17944, 17945, 17945, 17944, 17945, 17944, 17945, 17944, 17945]
INFO:absl:Generating split validation


Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteN53DWB/cnn_dailymail-validation.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteN53DWB/cnn_dailymail-validation.tfrecord. Shard lengths: [13368]
INFO:absl:Generating split test


Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteN53DWB/cnn_dailymail-test.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteN53DWB/cnn_dailymail-test.tfrecord. Shard lengths: [11490]
INFO:absl:Skipping computing stats for mode ComputeStatsMode.AUTO.
INFO:absl:Constructing tf.data.Dataset for split None, from /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0


Dataset cnn_dailymail downloaded and prepared to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0. Subsequent calls will reuse this data.


#Create keywords string as query

In [5]:
data = article

In [6]:
tfv = TfidfVectorizer(strip_accents='ascii', stop_words="english", sublinear_tf=True, max_df=0.8, min_df=5)
tfvalue = tfv.fit_transform(data)

In [7]:
def get_top_topic(tfvalues, top_number, words, center=4):
    row_number = tfvalues.shape[0]
    final_output = []
    for i in range(row_number):
        random_range = np.random.poisson(center)
        non_zero_index = tfvalue.getrow(i).nonzero()[1]
        top_index_in_non_zero_index = tfvalue[i, tfvalue.getrow(i).nonzero()[1]].toarray()[0].argsort()[:-top_number-1:-1]
        index = [non_zero_index[top_index_in_non_zero_index]]
        final_output.append(words[index][:random_range])
        
    return final_output

In [8]:
words = np.array(tfv.get_feature_names())


In [9]:
result = get_top_topic(tfvalue, 10, words)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if __name__ == '__main__':


In [10]:
keyword_string = [" ".join(list(words)) for words in result]

#Load model and split dataset

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
from torch.utils.data import DataLoader

In [13]:
train_article, test_article, train_traget, test_target, train_keyword, test_keyword = train_test_split(article, target, keyword_string, train_size=0.5)

In [14]:
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

100%|██████████| 245M/245M [00:12<00:00, 19.9MB/s]


# Evaluate before training

In [15]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

In [16]:
number_to_evaluate = 1000
number_of_keywords = 10
corpus = {str(i): test_article[i] for i in range(number_to_evaluate)}
relevant_docs = {str(i):str(i) for i in range(number_to_evaluate) }


queries = {str(i): test_keyword[i] for i in range(number_to_evaluate) }
evaluater = InformationRetrievalEvaluator(queries=queries, corpus=corpus, relevant_docs=relevant_docs, mrr_at_k=[1, 2, 3, 4, 5, 8, 10, 12])

In [17]:
model.evaluate(evaluater, output_path="before_finetuning")

0.13617579901711213

In [18]:
train_length = 10000
train_examples = [InputExample(texts=[train_article[i], train_keyword[i], train_traget[i]], label=1) for i in range(train_length)]
train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)

#Train

In [ ]:
train_loss = losses.BatchHardTripletLoss(model)

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=2, warmup_steps=0,  evaluator=evaluater, output_path="test7",
          optimizer_params = {'lr': 2e-1, 'eps': 1e-6, 'correct_bias': False},
            weight_decay = 0)

In [ ]:
model.evaluate(evaluater, output_path="before_finetuning")